In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes

In [27]:
import torch
torch.cuda.empty_cache()
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [28]:
from huggingface_hub import notebook_login
notebook_login()

In [44]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("./my_trained_model")
tokenizer = T5Tokenizer.from_pretrained("./my_trained_model")
model.to(device)


print("Model loaded successfully.")

Model loaded successfully.


In [83]:
import json
import pandas as pd
import re
import os

train_excel_path = "esgish.xlsx"

train_df = pd.read_excel(train_excel_path)
expected_columns = ["Esgish", "English"]
train_df.rename(columns=lambda x: x.strip(), inplace=True)

for col in expected_columns:
    assert col in train_df.columns, f"Missing expected column: {col}"
print("Training data loaded successfully.")
print(train_df.head())

train_data = []

for _, row in train_df.iterrows():
    train_data.append({
        "input": f"Translate to English: {row['Esgish']}",
        "output": row["English"]
    })

train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))
print("Training data processed successfully.")
print(train_dataset[0])

print("Test data loaded successfully.")
print(test_df.head())


Training data loaded successfully.
                                            Esgish  \
0             [CCANuclearPowerNewPlants] == 'True'   
1                      [CCANuclearPowerTPES] < '5'   
2                      [CCANuclearPowerTPES] > '0'   
3  [CCANuclearWeaponsSignatory] == 'Non-Signatory'   
4       [CEOEmployeePayDisclosedByCompany] > '0.5'   

                                             English  
0  Return all countries with a new power plant th...  
1  Return all countries with a Total Primary Ener...  
2  Return all countries with a Total Primary Ener...  
3  Return all countries that did not sign the Tre...  
4  Return all issuers with a ratio between CEO pa...  
Training data processed successfully.
Test data loaded successfully.
                                              Esgish  English
0  AND([CarbonRRPerformanceScore] > '0',[CarbonRR...      NaN
1                       [CEOSpecialGrantPct] > '0.5'      NaN
2  AND([CIQMarketCap] >= '10000000000',[oekomIndu...   

In [85]:
!pip install datasets
!pip install sentencepiece

from datasets import load_dataset, Dataset

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


def preprocess_function(examples):
    inputs = tokenizer(examples["input"], truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(examples["output"], max_length=128, truncation=True, padding="max_length")    
    model_inputs = {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels["input_ids"], 
    }
    

    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["input", "output"])
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=["input", "output"])
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=["input", "output"])
print(train_dataset[0])


Esgish: Translate to English: [CCANuclearPowerNewPlants] == 'True'
English: Return all countries with a new power plant that is under construction, planned for construction, or have been put into operation in the prior year.
---
Esgish: Translate to English: [CCANuclearPowerTPES] < '5'
English: Return all countries with a Total Primary Energy Supply less than 5%.
---
Esgish: Translate to English: [CCANuclearPowerTPES] > '0'
English: Return all countries with a Total Primary Energy Supply greater than 0%.
---
Esgish: Translate to English: [CCANuclearWeaponsSignatory] == 'Non-Signatory'
English: Return all countries that did not sign the Treaty on the Non-Proliferation of Nuclear Weapons.
---
Esgish: Translate to English: [CEOEmployeePayDisclosedByCompany] > '0.5'
English: Return all issuers with a ratio between CEO pay and employee pay of greater than 50%.
---


In [61]:
#split for evaluation

#split_dataset = train_dataset.train_test_split(test_size=0.1)
#train_dataset = split_dataset['train']
#eval_dataset = split_dataset['test']


In [62]:
!pip install accelerate -U
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, T5Tokenizer

training_args = TrainingArguments(
    output_dir="flan-t5-sql",
    eval_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    push_to_hub=False,
    report_to="none",
    remove_unused_columns=False,
    load_best_model_at_end=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer = tokenizer,
    data_collator=data_collator,

)

print("Training started...")

trainer.train()
print("training done")

test_results = trainer.evaluate(test_dataset)
print(f"Test Loss: {test_results['eval_loss']}")

model.save_pretrained("./my_trained_model")
tokenizer.save_pretrained("./my_trained_model")


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Training started...


Epoch,Training Loss,Validation Loss
1,No log,3.604892
2,No log,3.081428
3,No log,2.837015


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


training done


('./my_trained_model/tokenizer_config.json',
 './my_trained_model/special_tokens_map.json',
 './my_trained_model/spiece.model',
 './my_trained_model/added_tokens.json')

In [63]:
def translate_query(query):
    input_text = f"Translate to English: {query}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=128,
        num_beams=5,  
        early_stopping=True,
    )
    
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

user_query = "Find companies with high carbon emissions"
translated_query = translate_query(user_query)
print(f"Translated Query: {translated_query}")

Generated English queries for test data.
                                              Esgish  \
0  AND([CarbonRRPerformanceScore] > '0',[CarbonRR...   
1                       [CEOSpecialGrantPct] > '0.5'   
2  AND([CIQMarketCap] >= '10000000000',[oekomIndu...   

                                   Generated English  
0                      Performance Scores > '0', '0'  
1                                            QR code  
2  Warranty for the use of a 'CIQ Market Cap', 'O...  


In [64]:
output_test_path = "test_data_with_predictions.xlsx"
test_df.to_excel(output_test_path, index=False)

print(f"Results saved to {output_test_path}")


Results saved to test_data_with_predictions.xlsx
